In [14]:
import requests as rq
from bs4 import BeautifulSoup as BS
from selenium import webdriver
import io

def RemoveBlank(st):
    idx = 0;
    for a in st:
        if(a is '\n'):
            idx = idx + 1
        elif(a is '\t'):
            idx = idx + 1
        elif(a is ' '):
            idx = idx + 1
        else:
            break
    
    st = st[idx:]
            
    last = 0
    chk = 0
    for a in st:
        last = last + 1
        if(a is ' '):
            chk = chk + 1
            continue
        else:
            chk = 0
        
        if(chk >= 2):
            break
        elif(a is '\n'):
            break
        elif(a is '\t'):
            break
            
    st = st[:last]
    
    return st

def getskku():
    
    req = rq.get("https://www.skku.edu/skku/campus/skk_comm/notice01.do")
    req.encoding = "utf-8"
    
    html = req.text
    bsObj = BS(html,"html.parser")
    lists = bsObj.select(
        "tbody > tr > td.left > a"
    )
    
    result = list()
    for sentence in lists:
        result.append(sentence.text)
        
    idx = 0
    for s in result:
        result[idx] = RemoveBlank(s)
        idx = idx + 1
    
    new_notice = False
    with io.open("skku.txt",mode="r",encoding = 'utf-8') as f:
        line = f.readline()
        if line != result[0]:
            print("성균관대학교에 새로운 전체 공지사항")
            new_notice = True

    with io.open('skku.txt',mode = "w", encoding = "utf-8") as f:
        for s in result:
            f.write(s)
        
    print("성균관대학교 공지사항 읽어오기 완료.\n")
    
    return new_notice
        
def getsosc(CHROME_DIR):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    
    URL = "http://cs.skku.edu/news/recent/list"
    driver = webdriver.Chrome(CHROME_DIR, chrome_options=options)
    
    driver.get(URL)
    html = driver.page_source
    driver.close()

    soup = BS(html,"html.parser")
    notices = soup.select(
        "tbody > tr > td"
    )
    
    idx = 1
    result = list()
    
    while idx < 50:
        result.append(notices[idx].text)
        idx = idx + 5
    
    idx = 0
    for s in result:
        result[idx] = result[idx] + "\n"
        idx = idx + 1
        
    new_notice = False
    with io.open("sosc.txt",mode="r",encoding = 'utf-8') as f:
        line = f.readline()
        if line != result[0]:
            print("소프트웨어학과에 새로운 공지사항\n")
            new_notice = True
    
    with io.open('sosc.txt',mode = "w", encoding = "utf-8") as f:
        for s in result:
            f.write(s)
    
    print("소프트웨어학과 공지사항 읽어오기 완료.\n")
    return new_notice

def getIcam(CHROME_DIR, ID, pw):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    
    driver = webdriver.Chrome(CHROME_DIR, chrome_options = options)
    driver.get("http://www.icampus.ac.kr/front/main/MainAction.do?method=list")
    driver.find_element_by_name('uid').send_keys(ID)
    driver.find_element_by_name('pwd').send_keys(pw)
    driver.find_elements_by_xpath('//*[@id="mlogin01"]/div/a')
    driver.find_element_by_xpath('//*[@id="mlogin02"]/ul/li[1]/a').click()

    html = driver.page_source
    driver.close()

    soup = BS(html,"html.parser")
    lists = soup.findAll("a", {"href" : r"#"})
    
    result = list()
    for s in lists:
        result.append(s.text)
    
    idx = 0
    for s in result:
        result[idx] = RemoveBlank(s)
        idx = idx + 1

    where = result.index("과목공지")
    result = result[where - 1:]
    result = ' '.join(result).split()
    
    idx = 0
    for s in result:
        result[idx] = result[idx] + "\n"
        idx = idx + 1
    
    new_notice = False
    with io.open("Icam.txt",mode="r",encoding = 'utf-8') as f:
        cur = 0
        while True:
            line = f.readline()
            if not line: 
                break
            if line != result[cur]:
                print("I-campus에 새로운 공지사항\n")
                new_notice = True
                break
            cur = cur + 1
    
    with io.open('Icam.txt',mode = "w", encoding = "utf-8") as f:
        for s in result:
            f.write(s)
    
    print("I-campus 공지사항 읽어오기 완료.\n")
    return new_notice

with io.open('Info.txt',mode = "r", encoding = "utf-8") as f:
    ID = f.readline()
    pw = f.readline()
    CHROME_DIR = f.readline()
    
skku = getskku()
sosc = getsosc(CHROME_DIR)
icam = getIcam(CHROME_DIR, ID, pw)

if skku is True:
    driver_skku = webdriver.Chrome(CHROME_DIR)
    driver_skku.get("https://www.skku.edu/skku/campus/skk_comm/notice01.do")
if sosc is True:
    driver_sosc = webdriver.Chrome(CHROME_DIR)
    driver_sosc.get("http://cs.skku.edu/news/recent/list")
    
if icam is True:
    Idriver = webdriver.Chrome(CHROME_DIR)
    Idriver.get("http://www.icampus.ac.kr/front/main/MainAction.do?method=list")
    Idriver.find_element_by_name('uid').send_keys(ID)
    Idriver.find_element_by_name('pwd').send_keys(pw)
    Idriver.find_elements_by_xpath('//*[@id="mlogin01"]/div/a')
    Idriver.find_element_by_xpath('//*[@id="mlogin02"]/ul/li[1]/a').click()

성균관대학교 공지사항 읽어오기 완료.



C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:83: DeprecationWarning: use options instead of chrome_options


소프트웨어학과 공지사항 읽어오기 완료.



C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: DeprecationWarning: use options instead of chrome_options


I-campus에 새로운 공지사항

I-campus 공지사항 읽어오기 완료.

